In [ ]:
import psycopg2
from psycopg2 import Binary
from picamera2 import Picamera2, Preview
import time
import cv2
from flask import Flask, Response
from libcamera import controls
import requests
import json

app = Flask(__name__)
    
def getAllCars(url):
    url += "getAllCars"

    payload = json.dumps({
      "LotID": "c845a7d2-03c6-4dcd-a241-2db0b8905c1f"
    })
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    print(response.text)

def updateCar(url, carID, licensePlate, color, valid):
    url += "updateCar"

    payload = json.dumps({
      "CarID": carID,
      "license_plate": licensePlate,
      "color": {
        "String": color,
        "Valid": valid
      }
    })
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("PUT", url, headers=headers, data=payload)

    print(response.text)
    
def saveCar():
    url += "saveCar"

    payload = json.dumps({
      "license_plate": "621-ABC",
      "color": {
        "String": "Orange",
        "Valid": True
      },
      "LotID": "f24dacb0-9513-45d9-90f8-b07e5dee5271"
    })
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    print(response.text)
    
def deleteCar(url):
    url += "deleteCar"

    payload = json.dumps({
      "CarID": "9b5559cf-d277-4a52-b341-7d5831e597d0"
    })
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("DELETE", url, headers=headers, data=payload)

    print(response.text)
    
def getAllLots(url):
    url = "http:://ec2-3-143-172-128.us-east-2.compute.amazonaws.com:8080/getAllLots"

    payload = {}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    print(response.text)

def updateCar():
    url = "http:://ec2-3-143-172-128.us-east-2.compute.amazonaws.com:8080/saveCar?1=Honda"

    payload = json.dumps({
      "license_plate": "621-ABC",
      "color": {
        "String": "Orange",
        "Valid": True
      },
      "LotID": "f24dacb0-9513-45d9-90f8-b07e5dee5271"
    })
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    print(response.text)

def captureCar():
    # picam2 = Picamera2()
    camera_config = picam2.create_still_configuration(main={'size': (1920, 1080)})
    picam2.configure(camera_config)
    try:
        picam2.start()
        time.sleep(2) # zzz
        picam2.capture_file("test.jpg")
    except Exception as e:
        print(f"Camera smells like: {e}")

def captureFrames():
    picam2.close()
    picam2 = Picamera2()
    camera_config = picam2.create_video_configuration(controls={"FrameDurationLimits": (100000, 100000),
                                                                'NoiseReductionMode': controls.draft.NoiseReductionModeEnum.Fast},
                                                      main={"size": (1920, 1080)})
    picam2.configure(camera_config)
    picam2.start()
    i = 0
    while True:
        frame = picam2.capture_array()
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image\r\n\r\n' + frame + b'\r\n')
        i += 1
        print(i)

@app.route('/video_feed')
def video_feed():
    return Response(captureFrames(), mimetype='multipart/x-mixed-replace; boundary=frame')

def main():
    print("Opening secrets")
    f = open("/home/parkings/Desktop/TemporaryConnectionFolder/url.txt")
    url = str(f.readline()).strip() 
    
    getAllCars(url)
    
    app.run(host='0.0.0.0', port=5000)
    captureFrames()
    

if __name__ == "__main__":
    main()


Opening secrets
[]
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://10.136.81.94:5002
Press CTRL+C to quit
10.228.9.57 - - [05/Dec/2024 14:24:09] "GET /video_feed HTTP/1.1" 500 -
Error on request:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/werkzeug/serving.py", line 335, in run_wsgi
    execute(self.server.app)
  File "/usr/lib/python3/dist-packages/werkzeug/serving.py", line 324, in execute
    for data in application_iter:
  File "/usr/lib/python3/dist-packages/werkzeug/wsgi.py", line 500, in __next__
    return self._next()
           ^^^^^^^^^^^^
  File "/usr/lib/python3/dist-packages/werkzeug/wrappers/response.py", line 50, in _iter_encoded
    for item in iterable:
  File "/tmp/ipykernel_2868/804360261.py", line 120, in captureFrames
    picam2.close()
UnboundLocalError: cannot access local variable 'picam2' where it is not associated with a value
10.228.9.57 - - [05/Dec/2024 14:24:10] "GET /favicon.ico HTTP/1.1" 404 -
